In [216]:
import numpy as np
"""
Computes camera matrix given image and real-world coordinates.

    Args:
        real_XY: Each row corresponds to an actual point on the 2D plane.
        front_image: Each row is the pixel location in the front image (Z=0).
        back_image: Each row is the pixel location in the back image (Z=150).
    Returns:
        camera_matrix: The calibrated camera matrix (3x4 matrix).
"""
"I guess such corresponding points are hand-crafted"
real_XY = np.load('real_XY.npy')
front_image = np.load('front_image.npy')
back_image = np.load('back_image.npy')

#### 1. Perspective Camera Calibration

Deal with 11 DFs camera matrix with 12 parameters in total, solved through constraint minimization problem with SVD.

#### 2. Affine Camera Calibration

Deal with 8 DFs camera matrix with 8 parameters in total, solved with LSE on linear system.

In [167]:
# world coordinates: (M,3), image_coordinates: (M,2)
def calibrate_perspective_camera(world_coordinates, image_coordinates):
    wc,ic = world_coordinates, image_coordinates
    wc = np.pad(wc,[(0,0),(0,1)],mode='constant',constant_values=1.) # (M,4)
    pstack = np.repeat(wc, repeats=2, axis=0) # (2M,4)
    # scale even & odd rows with different values
    v1 = np.array([1,0])
    tmp1 = np.tile(v1,wc.shape[0])[:,np.newaxis] * pstack # (2M,4)
    v2 = np.array([0,1])
    tmp2 = np.tile(v2,wc.shape[0])[:,np.newaxis] * pstack # (2M,4)
    pix = ic.reshape(-1,1) # (2M,1)    
    tmp3 = - pix * pstack # (2M,4)
    P = np.concatenate((tmp1,tmp2,tmp3),axis=1) # (2M,12)
    # optimization problem solved with Singular Value Decomposition
    u,s,vh = np.linalg.svd(P)
    # take last column of V, or the last row of V.T: (12,)
    sol = vh[-1]
    # construct camera matrix M: (3,4)
    M = sol.reshape(3,4)
    return M

# affine camera model: Standard Least Squares
def calibrate_affine_camera(world_coordinates, image_coordinates):
    wc,ic = world_coordinates, image_coordinates
    wc = np.pad(wc,[(0,0),(0,1)],mode='constant',constant_values=1.) # (M,4)
    pstack = np.repeat(wc, repeats=2, axis=0) # (2M,4)
    # scale even & odd rows with different values
    v1 = np.array([1,0])
    tmp1 = np.tile(v1,wc.shape[0])[:,np.newaxis] * pstack # (2M,4)
    v2 = np.array([0,1])
    tmp2 = np.tile(v2,wc.shape[0])[:,np.newaxis] * pstack # (2M,4)
    P = np.concatenate((tmp1,tmp2),axis=1) # (2M,8)
    b = image_coordinates.reshape(-1,1) # (2M,1)
    m_ls = np.linalg.inv(P.T@P)@(P.T)@b # (8,1)
    # construct affine camera matrix
    M12 = m_ls.reshape(2,4) # (2,4)
    M3 = np.array([[0.,0.,0.,1.]]) # (1,4)
    M = np.concatenate((M12,M3),axis=0)
    return M

To generalize the usage, specify world_coordinates (-1,3) and image_coordinates (-1,2) for application of above function.

In [213]:
# world coordinate for first imageL (N,3)
wc1 = np.pad(real_XY,[(0,0),(0,1)],mode='constant',constant_values=0.)
# world coordinate for second image: (N,3)
wc2 = np.pad(real_XY,[(0,0),(0,1)],mode='constant',constant_values=150.)
# world coordinates (3D) and image coordinate (2D) construction
world_coordinates = np.concatenate((wc1,wc2),axis=0) # (2N,3)
image_coordinates = np.concatenate((front_image,back_image)) # (2N,2)

# fit the perspective camera model
M0 = calibrate_perspective_camera(world_coordinates, image_coordinates)
M1 = calibrate_affine_camera(world_coordinates, image_coordinates)
print('Calibrated Perspective Camera model is:\n',M0)
print('\n Calibrated Affine Camera model is:\n',M1)

Calibrated Perspective Camera model is:
 [[ 3.86081985e-03 -1.14839115e-04  8.75272791e-04  9.46068598e-01]
 [ 3.42814033e-04  3.92515324e-03 -7.51681648e-04  3.23835234e-01]
 [-6.97426063e-08  8.23266292e-08 -1.33752672e-08  7.29214554e-03]]

 Calibrated Affine Camera model is:
 [[ 5.31276507e-01 -1.80886074e-02  1.20509667e-01  1.29720641e+02]
 [ 4.84975447e-02  5.36366401e-01 -1.02675222e-01  4.43879607e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


#### 3. RMS Error

In [214]:
def rms_error(world_coordinates, image_coordinates, camera_matrx):
    wc,ic,M = world_coordinates, image_coordinates, camera_matrx
    # obtain predicted image coordinate
    wc_proj = np.pad(wc,[(0,0),(0,1)],mode='constant',constant_values=1) # (M,4)
    pred_ic_proj = (M @ wc_proj[:,:,np.newaxis])[:,:,0] # (M,3)
    pred_ic = pred_ic_proj[:,:2]/pred_ic_proj[:,2:] # (M,2)
    # compute RMS
    sq_diff = np.sum((pred_ic - ic)**2,axis=1)
    rms = np.sqrt(np.mean(sq_diff))
    return rms

print('RMS for calibrated perspective camera model is:',rms_error(world_coordinates, image_coordinates, M0))
print('RMS for calibrated affine camera model is:',rms_error(world_coordinates, image_coordinates, M1))

RMS for calibrated perspective camera model is: 0.9916933372299293
RMS for calibrated affine camera model is: 0.9938304832798437


#### 4. Vanishing points